In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/cross-domain-recommendation/dataset/leak_stats.py
/kaggle/input/cross-domain-recommendation/dataset/read_file.py
/kaggle/input/cross-domain-recommendation/dataset/Food-Kitchen/validdata_new.txt
/kaggle/input/cross-domain-recommendation/dataset/Food-Kitchen/Blist.txt
/kaggle/input/cross-domain-recommendation/dataset/Food-Kitchen/testdata_new.txt
/kaggle/input/cross-domain-recommendation/dataset/Food-Kitchen/traindata.txt
/kaggle/input/cross-domain-recommendation/dataset/Food-Kitchen/validdata.txt
/kaggle/input/cross-domain-recommendation/dataset/Food-Kitchen/Alist.txt
/kaggle/input/cross-domain-recommendation/dataset/Food-Kitchen/testdata.txt
/kaggle/input/cross-domain-recommendation/dataset/Food-Kitchen/traindata_new.txt
/kaggle/input/cross-domain-recommendation/dataset/Food-Kitchen/userlist.txt
/kaggle/input/cross-domain-recommendation/dataset/Movie-Book/validdata_new.txt
/kaggle/input/cross-domain-recommendation/dataset/Movie-Book/Blist.txt
/kaggle/input/cross-domain-re

# GNN Model

In [2]:
import torch.nn as nn
import torch.nn.functional as F
import math
import torch
import torch.nn as nn
from torch.nn.modules.module import Module
import numpy as np

In [3]:
class GCNLayer(nn.Module):
    """
        GCN Module layer
    """
    def __init__(self, opt):
        super(GCNLayer, self).__init__()
        self.opt=opt
        self.dropout = opt["dropout"]
        self.layer_number = opt["GNN"]

        self.encoder = []
        for i in range(self.layer_number):
            self.encoder.append(GNN(
                nfeat=opt["hidden_units"],
                nhid=opt["hidden_units"],
                dropout=opt["dropout"],
                alpha=opt["leakey"]))

        self.encoder = nn.ModuleList(self.encoder)

    def forward(self, fea, adj):
        learn_fea = fea
        tmp_fea = fea
        for layer in self.encoder:
            learn_fea = F.dropout(learn_fea, self.dropout, training=self.training)
            learn_fea = layer(learn_fea, adj)
            tmp_fea = tmp_fea + learn_fea
        return tmp_fea / (self.layer_number + 1)


class GNN(nn.Module):
    def __init__(self, nfeat, nhid, dropout, alpha):
        super(GNN, self).__init__()
        self.gc1 = GraphConvolution(nfeat, nhid)
        self.dropout = dropout
        self.leakyrelu = nn.LeakyReLU(alpha)

    def forward(self, x, adj):
        x = self.gc1(x, adj)
        return x

In [4]:
class GraphConvolution(Module):
    def __init__(self, in_features, out_features, bias=True):
        super(GraphConvolution, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.weight = nn.Parameter(torch.FloatTensor(in_features, out_features))
        # self.weight = self.glorot_init(in_features, out_features)
        if bias:
            self.bias = nn.Parameter(torch.FloatTensor(out_features))
        else:
            self.register_parameter('bias', None)
        self.reset_parameters()

    def reset_parameters(self):
        stdv = 1. / math.sqrt(self.weight.size(1))
        self.weight.data.uniform_(-stdv, stdv)
        if self.bias is not None:
            self.bias.data.uniform_(-stdv, stdv)

    def glorot_init(self, input_dim, output_dim):
        init_range = np.sqrt(6.0 / (input_dim + output_dim))
        initial = torch.rand(input_dim, output_dim) * 2 * init_range - init_range
        return nn.Parameter(initial / 2)

    def forward(self, input, adj):
        support = input
        output = torch.spmm(adj, support)
        if self.bias is not None:
            return output + self.bias
        else:
            return output

    def __repr__(self):
        return self.__class__.__name__ + ' (' \
               + str(self.in_features) + ' -> ' \
               + str(self.out_features) + ')'

# Model

In [5]:
import numpy as np
import torch
import torch.nn as nn

In [6]:
class Discriminator(torch.nn.Module):
    def __init__(self, n_in,n_out):
        super(Discriminator, self).__init__()
        self.f_k = nn.Bilinear(n_in, n_out, 1)
        for m in self.modules():
            self.weights_init(m)

    def weights_init(self, m):
        if isinstance(m, nn.Bilinear):
            torch.nn.init.xavier_uniform_(m.weight.data)
            if m.bias is not None:
                m.bias.data.fill_(0.0)

    def forward(self, S, node, s_bias=None):
        score = self.f_k(node, S)
        if s_bias is not None:
            score += s_bias
        return score

In [7]:
class PointWiseFeedForward(torch.nn.Module):
    def __init__(self, hidden_units, dropout_rate):

        super(PointWiseFeedForward, self).__init__()
        self.gru1 = torch.nn.GRU(hidden_units, hidden_units, batch_first=True)
#         self.conv1 = torch.nn.Conv1d(hidden_units, hidden_units, kernel_size=1)
        self.dropout1 = torch.nn.Dropout(p=dropout_rate)
        self.relu = torch.nn.ReLU()
        self.gru2 = torch.nn.GRU(hidden_units, hidden_units, batch_first=True)
#         self.conv2 = torch.nn.Conv1d(hidden_units, hidden_units, kernel_size=1)
        self.dropout2 = torch.nn.Dropout(p=dropout_rate)

    def forward(self, inputs):
#         print(f"Shape of inputs {inputs.shape}")
        outputs = self.dropout2(self.gru2(self.relu(self.dropout1(self.gru1(inputs)[0])))[0])
#         outputs = outputs.transpose(-1, -2) 
        outputs += inputs
        return outputs

In [8]:
class ATTENTION(torch.nn.Module):
    def __init__(self, opt):
        super(ATTENTION, self).__init__()
        self.opt = opt
        self.emb_dropout = torch.nn.Dropout(p=self.opt["dropout"])
        self.pos_emb = torch.nn.Embedding(self.opt["maxlen"], self.opt["hidden_units"], padding_idx=0)  # TO IMPROVE
        # TODO: loss += args.l2_emb for regularizing embedding vectors during training
        # https://stackoverflow.com/questions/42704283/adding-l1-l2-regularization-in-pytorch
        self.attention_layernorms = torch.nn.ModuleList() # to be Q for self-attention
        self.attention_layers = torch.nn.ModuleList()
        self.forward_layernorms = torch.nn.ModuleList()
        self.forward_layers = torch.nn.ModuleList()

        self.last_layernorm = torch.nn.LayerNorm(self.opt["hidden_units"], eps=1e-8)

        for _ in range(self.opt["num_blocks"]):
            new_attn_layernorm = torch.nn.LayerNorm(self.opt["hidden_units"], eps=1e-8)
            self.attention_layernorms.append(new_attn_layernorm)

            new_attn_layer =  torch.nn.MultiheadAttention(self.opt["hidden_units"],
                                                            self.opt["num_heads"],
                                                            self.opt["dropout"])
            self.attention_layers.append(new_attn_layer)

            new_fwd_layernorm = torch.nn.LayerNorm(self.opt["hidden_units"], eps=1e-8)
            self.forward_layernorms.append(new_fwd_layernorm)

            new_fwd_layer = PointWiseFeedForward(self.opt["hidden_units"], self.opt["dropout"])
            self.forward_layers.append(new_fwd_layer)

    def forward(self, seqs_data, seqs, position):
        # positions = np.tile(np.array(range(log_seqs.shape[1])), [log_seqs.shape[0], 1])
        seqs += self.pos_emb(position)
        seqs = self.emb_dropout(seqs)

        timeline_mask = torch.BoolTensor(seqs_data.cpu() == self.opt["itemnum"] - 1)
        if self.opt["cuda"]:
            timeline_mask = timeline_mask.cuda()
        seqs *= ~timeline_mask.unsqueeze(-1) # broadcast in last dim

        tl = seqs.shape[1] # time dim len for enforce causality
        attention_mask = ~torch.tril(torch.ones((tl, tl), dtype=torch.bool))
        if self.opt["cuda"]:
            attention_mask = attention_mask.cuda()

        for i in range(len(self.attention_layers)):
            seqs = torch.transpose(seqs, 0, 1)
            Q = self.attention_layernorms[i](seqs)
            mha_outputs, _ = self.attention_layers[i](Q, seqs, seqs,
                                            attn_mask=attention_mask)
                                            # key_padding_mask=timeline_mask
                                            # need_weights=False) this arg do not work?
            seqs = Q + mha_outputs
            seqs = torch.transpose(seqs, 0, 1)

            seqs = self.forward_layernorms[i](seqs)
            seqs = self.forward_layers[i](seqs)
            seqs *=  ~timeline_mask.unsqueeze(-1)

        log_feats = self.last_layernorm(seqs) # (U, T, C) -> (U, -1, C)

        return log_feats

In [9]:
class C2DSR(torch.nn.Module):
    def __init__(self, opt, adj, adj_single):
        super(C2DSR, self).__init__()
        self.opt = opt
        self.item_emb_X = torch.nn.Embedding(self.opt["itemnum"], self.opt["hidden_units"],
                                           padding_idx=self.opt["itemnum"] - 1)
        self.item_emb_Y = torch.nn.Embedding(self.opt["itemnum"], self.opt["hidden_units"],
                                           padding_idx=self.opt["itemnum"] - 1)
        self.item_emb = torch.nn.Embedding(self.opt["itemnum"], self.opt["hidden_units"],
                                           padding_idx=self.opt["itemnum"] - 1)
        self.GNN_encoder_X = GCNLayer(opt)
        self.GNN_encoder_Y = GCNLayer(opt)
        self.GNN_encoder = GCNLayer(opt)
        self.adj = adj
        self.adj_single = adj_single

        self.D_X = Discriminator(self.opt["hidden_units"], self.opt["hidden_units"])
        self.D_Y = Discriminator(self.opt["hidden_units"], self.opt["hidden_units"])

        self.lin_X = nn.Linear(self.opt["hidden_units"], self.opt["source_item_num"])
        self.lin_Y = nn.Linear(self.opt["hidden_units"], self.opt["target_item_num"])
        self.lin_PAD = nn.Linear(self.opt["hidden_units"], 1)
        self.encoder = ATTENTION(opt)
        self.encoder_X = ATTENTION(opt)
        self.encoder_Y = ATTENTION(opt)

        self.source_item_index = torch.arange(0, self.opt["source_item_num"], 1)
        self.target_item_index = torch.arange(self.opt["source_item_num"], self.opt["source_item_num"]+self.opt["target_item_num"], 1)
        self.item_index = torch.arange(0, self.opt["itemnum"], 1)
        if self.opt["cuda"]:
            self.source_item_index = self.source_item_index.cuda()
            self.target_item_index = self.target_item_index.cuda()
            self.item_index = self.item_index.cuda()

    def my_index_select_embedding(self, memory, index):
        tmp = list(index.size()) + [-1]
        index = index.view(-1)
        ans = memory(index)
        ans = ans.view(tmp)
        return ans

    def my_index_select(self, memory, index):
        tmp = list(index.size()) + [-1]
        index = index.view(-1)
        ans = torch.index_select(memory, 0, index)
        ans = ans.view(tmp)
        return ans

    def graph_convolution(self):
        fea = self.my_index_select_embedding(self.item_emb, self.item_index)
        fea_X = self.my_index_select_embedding(self.item_emb_X, self.item_index)
        fea_Y = self.my_index_select_embedding(self.item_emb_Y, self.item_index)

        self.cross_emb = self.GNN_encoder(fea, self.adj)
        self.single_emb_X = self.GNN_encoder_X(fea_X, self.adj_single)
        self.single_emb_Y = self.GNN_encoder_Y(fea_Y, self.adj_single)

    def forward(self, o_seqs, x_seqs, y_seqs, position, x_position, y_position):
        seqs = self.my_index_select(self.cross_emb, o_seqs) + self.item_emb(o_seqs)
        seqs *= self.item_emb.embedding_dim ** 0.5
        seqs_fea = self.encoder(o_seqs, seqs, position)

        seqs = self.my_index_select(self.single_emb_X, x_seqs) + self.item_emb_X(x_seqs)
        seqs *= self.item_emb.embedding_dim ** 0.5
        x_seqs_fea = self.encoder_X(x_seqs, seqs, x_position)

        seqs = self.my_index_select(self.single_emb_Y, y_seqs) + self.item_emb_Y(y_seqs)
        seqs *= self.item_emb.embedding_dim ** 0.5
        y_seqs_fea = self.encoder_Y(y_seqs, seqs, y_position)

        return seqs_fea, x_seqs_fea, y_seqs_fea

    def false_forward(self, false_seqs, position):
        seqs = self.my_index_select(self.cross_emb, false_seqs) + self.item_emb(false_seqs)
        seqs *= self.item_emb.embedding_dim ** 0.5
        false_seqs_fea = self.encoder(false_seqs, seqs, position)
        return false_seqs_fea

# Graph Maker

In [10]:
import numpy as np
import random
import scipy.sparse as sp
import torch
import codecs
import json
import copy

In [11]:
def normalize(mx):
    """Row-normalize sparse matrix"""
    rowsum = np.array(mx.sum(1))
    r_inv = np.power(rowsum, -1).flatten()
    r_inv[np.isinf(r_inv)] = 0.
    r_mat_inv = sp.diags(r_inv)
    mx = r_mat_inv.dot(mx)
    return mx

In [12]:
def sparse_mx_to_torch_sparse_tensor(sparse_mx):
    """Convert a scipy sparse matrix to a torch sparse tensor."""
    sparse_mx = sparse_mx.tocoo().astype(np.float32)
    indices = torch.from_numpy(
        np.vstack((sparse_mx.row, sparse_mx.col)).astype(np.int64))
    values = torch.from_numpy(sparse_mx.data)
    shape = torch.Size(sparse_mx.shape)
    return torch.sparse.FloatTensor(indices, values, shape)

In [13]:
class GraphMaker(object):
    def __init__(self, opt, filename):
        self.opt = opt
        self.user = set()
        self.item = set()
        train_data = []

        def takeSecond(elem):
            return elem[1]
        with codecs.open(filename, "r", encoding="utf-8") as infile:
            for id, line in enumerate(infile):
                res = []
                line = line.strip().split("\t")[2:]
                for w in line:
                    w = w.split("|")
                    res.append((int(w[0]), int(w[1])))
                res.sort(key=takeSecond)
                res_2 = []
                for r in res:
                    res_2.append(r[0])
                train_data.append(res_2)

        self.raw_data = train_data
        self.adj, self.adj_single = self.preprocess(train_data, opt)

    def preprocess(self,data,opt):

        VV_edges = []
        VV_edges_single = []

        real_adj = {}

        for seq in data:
            source = -1
            target = -1
            pre = -1
            for d in seq:
                if d not in real_adj:
                    real_adj[d] = set()
                if d < self.opt["source_item_num"]:
                    if source is not -1:
                        if d in real_adj[source]:
                            continue
                        else:
                            VV_edges_single.append([source, d])
                    source = d

                else :
                    if target is not -1:
                        if d in real_adj[target]:
                            continue
                        else:
                            VV_edges_single.append([target, d])
                    target = d

                if pre is not -1:
                    if d in real_adj[pre]:
                        continue
                    VV_edges.append([pre, d])
                pre=d

        VV_edges = np.array(VV_edges)
        VV_edges_single = np.array(VV_edges_single)
        adj = sp.coo_matrix((np.ones(VV_edges.shape[0]), (VV_edges[:, 0], VV_edges[:, 1])),
                               shape=(opt["itemnum"], opt["itemnum"]),
                               dtype=np.float32)
        adj_single = sp.coo_matrix((np.ones(VV_edges_single.shape[0]), (VV_edges_single[:, 0], VV_edges_single[:, 1])),shape=(opt["itemnum"], opt["itemnum"]),dtype=np.float32)

        adj = normalize(adj)
        adj_single = normalize(adj_single)
        adj = sparse_mx_to_torch_sparse_tensor(adj)
        adj_single = sparse_mx_to_torch_sparse_tensor(adj_single)

        print("real graph loaded!")
        return adj, adj_single

<>:41: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:49: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:56: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:41: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:49: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:56: SyntaxWarning: "is not" with a literal. Did you mean "!="?
/tmp/ipykernel_35/4294224670.py:41: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if source is not -1:
/tmp/ipykernel_35/4294224670.py:49: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if target is not -1:
/tmp/ipykernel_35/4294224670.py:56: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if pre is not -1:


# Helper

In [14]:
import os
import json
import sys

In [15]:
def check_dir(d):
    if not os.path.exists(d):
        print("Directory {} does not exist. Exit.".format(d))
        exit(1)

In [16]:
def check_files(files):
    for f in files:
        if f is not None and not os.path.exists(f):
            print("File {} does not exist. Exit.".format(f))
            exit(1)

In [17]:
def ensure_dir(d, verbose=True):
    if not os.path.exists(d):
        if verbose:
            print("Directory {} do not exist; creating...".format(d))
        os.makedirs(d)

In [18]:
def save_config(config, path, verbose=True):
    with open(path, 'w') as outfile:
        json.dump(config, outfile, indent=2)
    if verbose:
        print("Config saved to file {}".format(path))
    return config

In [19]:
def load_config(path, verbose=True):
    with open(path) as f:
        config = json.load(f)
    if verbose:
        print("Config loaded from file {}".format(path))
    return config

In [20]:
def print_config(config):
    info = "Running with the following configs:\n"
    for k, v in config.items():
        info += "\t{} : {}\n".format(k, str(v))
    print("\n" + info + "\n")
    return

In [21]:
class FileLogger(object):
    """
    A file logger that opens the file periodically and write to it.
    """

    def __init__(self, filename, header=None):
        self.filename = filename
        if os.path.exists(filename):
            # remove the old file
            os.remove(filename)
        if header is not None:
            with open(filename, 'w') as out:
                print(header, file=out)

    def log(self, message):
        with open(self.filename, 'a') as out:
            print(message, file=out)

# Loader

In [22]:
import json
import random
import torch
import numpy as np
import codecs
import copy
import pdb

In [23]:
class DataLoader(object):
    """
    Load data from json files, preprocess and prepare batches.
    """
    def __init__(self, filename, batch_size, opt, evaluation):
        self.batch_size = batch_size
        self.opt = opt
        self.eval = evaluation
        self.filename  = filename
        # ************* item_id *****************
        #/kaggle/input/cross-domain-recommendation
        opt["source_item_num"] = self.read_item("/kaggle/input/cross-domain-recommendation/dataset/" + filename + "/Alist.txt")
        opt["target_item_num"] = self.read_item("/kaggle/input/cross-domain-recommendation/dataset/" + filename + "/Blist.txt")

        # ************* sequential data *****************

        source_train_data = "/kaggle/input/cross-domain-recommendation/dataset/" + filename + "/traindata_new.txt"
        source_valid_data = "/kaggle/input/cross-domain-recommendation/dataset/" + filename + "/validdata_new.txt"
        source_test_data = "/kaggle/input/cross-domain-recommendation/dataset/" + filename + "/testdata_new.txt"

        if evaluation < 0:
            self.train_data = self.read_train_data(source_train_data)
            data = self.preprocess()
        elif evaluation == 2:
            self.test_data = self.read_test_data(source_valid_data)
            data = self.preprocess_for_predict()
        else :
            self.test_data = self.read_test_data(source_test_data)
            data = self.preprocess_for_predict()

        # shuffle for training
        if evaluation == -1:
            indices = list(range(len(data)))
            random.shuffle(indices)
            data = [data[i] for i in indices]
            if batch_size > len(data):
                batch_size = len(data)
                self.batch_size = batch_size
            if len(data)%batch_size != 0:
                data += data[:batch_size]
            data = data[: (len(data)//batch_size) * batch_size]
        else :
            batch_size = 2048
        self.num_examples = len(data)

        # chunk into batches
        data = [data[i:i+batch_size] for i in range(0, len(data), batch_size)]
        self.data = data

    def read_item(self, fname):
        item_number = 0
        with codecs.open(fname, "r", encoding="utf-8") as fr:
            for line in fr:
                item_number += 1
        return item_number

    def read_train_data(self, train_file):
        def takeSecond(elem):
            return elem[1]
        with codecs.open(train_file, "r", encoding="utf-8") as infile:
            train_data = []
            for id, line in enumerate(infile):
                res = []

                line = line.strip().split("\t")[2:]
                for w in line:
                    w = w.split("|")
                    res.append((int(w[0]), int(w[1])))
                res.sort(key=takeSecond)
                res_2 = []
                for r in res:
                    res_2.append(r[0])
                train_data.append(res_2)

        return train_data

    def read_test_data(self, test_file):
        def takeSecond(elem):
            return elem[1]
        with codecs.open(test_file, "r", encoding="utf-8") as infile:
            test_data = []
            for id, line in enumerate(infile):
                res = []
                line = line.strip().split("\t")[2:]
                for w in line:
                    w = w.split("|")
                    res.append((int(w[0]), int(w[1])))

                res.sort(key=takeSecond)

                res_2 = []
                for r in res[:-1]:
                    res_2.append(r[0])

                if res[-1][0] >= self.opt["source_item_num"]: # denoted the corresponding validation/test entry
                    test_data.append([res_2, 1, res[-1][0]])
                else :
                    test_data.append([res_2, 0, res[-1][0]])
        return test_data

    def preprocess_for_predict(self):

        if "Enter" in self.filename:
            max_len = 30
            self.opt["maxlen"] = 30
        else:
            max_len = 15
            self.opt["maxlen"] = 15

        processed=[]
        for d in self.test_data: # the pad is needed! but to be careful.
            position = list(range(len(d[0])+1))[1:]

            xd = []
            xcnt = 1
            x_position = []

            yd = []
            ycnt = 1
            y_position = []

            for w in d[0]:
                if w < self.opt["source_item_num"]:
                    xd.append(w)
                    x_position.append(xcnt)
                    xcnt += 1
                    yd.append(self.opt["source_item_num"] + self.opt["target_item_num"])
                    y_position.append(0)

                else:
                    xd.append(self.opt["source_item_num"] + self.opt["target_item_num"])
                    x_position.append(0)
                    yd.append(w)
                    y_position.append(ycnt)
                    ycnt += 1


            if len(d[0]) < max_len:
                position = [0] * (max_len - len(d[0])) + position
                x_position = [0] * (max_len - len(d[0])) + x_position
                y_position = [0] * (max_len - len(d[0])) + y_position

                xd = [self.opt["source_item_num"] + self.opt["target_item_num"]] * (max_len - len(d[0])) + xd
                yd = [self.opt["source_item_num"] + self.opt["target_item_num"]] * (max_len - len(d[0])) + yd
                seq = [self.opt["source_item_num"] + self.opt["target_item_num"]]*(max_len - len(d[0])) + d[0]


            x_last = -1
            for id in range(len(x_position)):
                id += 1
                if x_position[-id]:
                    x_last = -id
                    break

            y_last = -1
            for id in range(len(y_position)):
                id += 1
                if y_position[-id]:
                    y_last = -id
                    break

            negative_sample = []
            for i in range(999):
                while True:
                    if d[1] : # in Y domain, the validation/test negative samples
                        sample = random.randint(0, self.opt["target_item_num"] - 1)
                        if sample != d[2] - self.opt["source_item_num"]:
                            negative_sample.append(sample)
                            break
                    else : # in X domain, the validation/test negative samples
                        sample = random.randint(0, self.opt["source_item_num"] - 1)
                        if sample != d[2]:
                            negative_sample.append(sample)
                            break


            if d[1]:
                processed.append([seq, xd, yd, position, x_position, y_position, x_last, y_last, d[1], d[2]-self.opt["source_item_num"], negative_sample])
            else:
                processed.append([seq, xd, yd, position, x_position, y_position, x_last, y_last, d[1],
                                  d[2], negative_sample])
        return processed

    def preprocess(self):

        def myprint(a):
            for i in a:
                print("%6d" % i, end="")
            print("")
        """ Preprocess the data and convert to ids. """
        processed = []


        if "Enter" in self.filename:
            max_len = 30
            self.opt["maxlen"] = 30
        else:
            max_len = 15
            self.opt["maxlen"] = 15

        for d in self.train_data: # the pad is needed! but to be careful.

            ground = copy.deepcopy(d)[1:]


            share_x_ground = []
            share_x_ground_mask = []
            share_y_ground = []
            share_y_ground_mask = []
            for w in ground:
                if w < self.opt["source_item_num"]:
                    share_x_ground.append(w)
                    share_x_ground_mask.append(1)
                    share_y_ground.append(self.opt["target_item_num"])
                    share_y_ground_mask.append(0)
                else:
                    share_x_ground.append(self.opt["source_item_num"])
                    share_x_ground_mask.append(0)
                    share_y_ground.append(w - self.opt["source_item_num"])
                    share_y_ground_mask.append(1)


            d = d[:-1]  # delete the ground truth
            position = list(range(len(d)+1))[1:]
            ground_mask = [1] * len(d)



            xd = []
            xcnt = 1
            x_position = []


            yd = []
            ycnt = 1
            y_position = []

            corru_x = []
            corru_y = []

            for w in d:
                if w < self.opt["source_item_num"]:
                    corru_x.append(w)
                    xd.append(w)
                    x_position.append(xcnt)
                    xcnt += 1
                    corru_y.append(random.randint(0, self.opt["source_item_num"] - 1))
                    yd.append(self.opt["source_item_num"] + self.opt["target_item_num"])
                    y_position.append(0)

                else:
                    corru_x.append(random.randint(self.opt["source_item_num"], self.opt["source_item_num"] + self.opt["target_item_num"] - 1))
                    xd.append(self.opt["source_item_num"] + self.opt["target_item_num"])
                    x_position.append(0)
                    corru_y.append(w)
                    yd.append(w)
                    y_position.append(ycnt)
                    ycnt += 1

            now = -1
            x_ground = [self.opt["source_item_num"]] * len(xd) # caution!
            x_ground_mask = [0] * len(xd)
            for id in range(len(xd)):
                id+=1
                if x_position[-id]:
                    if now == -1:
                        now = xd[-id]
                        if ground[-1] < self.opt["source_item_num"]:
                            x_ground[-id] = ground[-1]
                            x_ground_mask[-id] = 1
                        else:
                            xd[-id] = self.opt["source_item_num"] + self.opt["target_item_num"]
                            x_position[-id] = 0
                    else:
                        x_ground[-id] = now
                        x_ground_mask[-id] = 1
                        now = xd[-id]
            if sum(x_ground_mask) == 0:
                print("pass sequence x")
                continue

            now = -1
            y_ground = [self.opt["target_item_num"]] * len(yd) # caution!
            y_ground_mask = [0] * len(yd)
            for id in range(len(yd)):
                id+=1
                if y_position[-id]:
                    if now == -1:
                        now = yd[-id] - self.opt["source_item_num"]
                        if ground[-1] > self.opt["source_item_num"]:
                            y_ground[-id] = ground[-1] - self.opt["source_item_num"]
                            y_ground_mask[-id] = 1
                        else:
                            yd[-id] = self.opt["source_item_num"] + self.opt["target_item_num"]
                            y_position[-id] = 0
                    else:
                        y_ground[-id] = now
                        y_ground_mask[-id] = 1
                        now = yd[-id] - self.opt["source_item_num"]
            if sum(y_ground_mask) == 0:
                print("pass sequence y")
                continue

            if len(d) < max_len:
                position = [0] * (max_len - len(d)) + position
                x_position = [0] * (max_len - len(d)) + x_position
                y_position = [0] * (max_len - len(d)) + y_position

                ground = [self.opt["source_item_num"] + self.opt["target_item_num"]] * (max_len - len(d)) + ground
                share_x_ground = [self.opt["source_item_num"]] * (max_len - len(d)) + share_x_ground
                share_y_ground = [self.opt["target_item_num"]] * (max_len - len(d)) + share_y_ground
                x_ground = [self.opt["source_item_num"]] * (max_len - len(d)) + x_ground
                y_ground = [self.opt["target_item_num"]] * (max_len - len(d)) + y_ground

                ground_mask = [0] * (max_len - len(d)) + ground_mask
                share_x_ground_mask = [0] * (max_len - len(d)) + share_x_ground_mask
                share_y_ground_mask = [0] * (max_len - len(d)) + share_y_ground_mask
                x_ground_mask = [0] * (max_len - len(d)) + x_ground_mask
                y_ground_mask = [0] * (max_len - len(d)) + y_ground_mask

                corru_x = [self.opt["source_item_num"] + self.opt["target_item_num"]] * (max_len - len(d)) + corru_x
                corru_y = [self.opt["source_item_num"] + self.opt["target_item_num"]] * (max_len - len(d)) + corru_y
                xd = [self.opt["source_item_num"] + self.opt["target_item_num"]] * (max_len - len(d)) + xd
                yd = [self.opt["source_item_num"] + self.opt["target_item_num"]] * (max_len - len(d)) + yd
                d = [self.opt["source_item_num"] + self.opt["target_item_num"]] * (max_len - len(d)) + d
            else:
                print("pass")

            processed.append([d, xd, yd, position, x_position, y_position, ground, share_x_ground, share_y_ground, x_ground, y_ground, ground_mask, share_x_ground_mask, share_y_ground_mask, x_ground_mask, y_ground_mask, corru_x, corru_y])
        return processed

    def __len__(self):
        return len(self.data)

    def __getitem__(self, key):
        """ Get a batch with index. """
        if not isinstance(key, int):
            raise TypeError
        if key < 0 or key >= len(self.data):
            raise IndexError
        batch = self.data[key]
        batch_size = len(batch)
        if self.eval!=-1:
            batch = list(zip(*batch))
            return (torch.LongTensor(batch[0]), torch.LongTensor(batch[1]), torch.LongTensor(batch[2]), torch.LongTensor(batch[3]),torch.LongTensor(batch[4]), torch.LongTensor(batch[5]), torch.LongTensor(batch[6]), torch.LongTensor(batch[7]),torch.LongTensor(batch[8]), torch.LongTensor(batch[9]), torch.LongTensor(batch[10]))
        else :
            batch = list(zip(*batch))

            return (torch.LongTensor(batch[0]), torch.LongTensor(batch[1]), torch.LongTensor(batch[2]), torch.LongTensor(batch[3]),torch.LongTensor(batch[4]), torch.LongTensor(batch[5]), torch.LongTensor(batch[6]), torch.LongTensor(batch[7]),torch.LongTensor(batch[8]), torch.LongTensor(batch[9]), torch.LongTensor(batch[10]), torch.LongTensor(batch[11]), torch.LongTensor(batch[12]), torch.LongTensor(batch[13]), torch.LongTensor(batch[14]), torch.LongTensor(batch[15]), torch.LongTensor(batch[16]), torch.LongTensor(batch[17]))

    def __iter__(self):
        for i in range(self.__len__()):
            yield self.__getitem__(i)

# Torch Utils

In [24]:
import torch
from torch import nn, optim
from torch.optim.optimizer import Optimizer

In [25]:
class MyAdagrad(Optimizer):
    """My modification of the Adagrad optimizer that allows to specify an initial
    accumulater value. This mimics the behavior of the default Adagrad implementation
    in Tensorflow. The default PyTorch Adagrad uses 0 for initial acculmulator value.

    Arguments:
        params (iterable): iterable of parameters to optimize or dicts defining
            parameter groups
        lr (float, optional): learning rate (default: 1e-2)
        lr_decay (float, optional): learning rate decay (default: 0)
        init_accu_value (float, optional): initial accumulater value.
        weight_decay (float, optional): weight decay (L2 penalty) (default: 0)
    """

    def __init__(self, params, lr=1e-2, lr_decay=0, init_accu_value=0.1, weight_decay=0):
        defaults = dict(lr=lr, lr_decay=lr_decay, init_accu_value=init_accu_value, \
                weight_decay=weight_decay)
        super(MyAdagrad, self).__init__(params, defaults)

        for group in self.param_groups:
            for p in group['params']:
                state = self.state[p]
                state['step'] = 0
                state['sum'] = torch.ones(p.data.size()).type_as(p.data) *\
                        init_accu_value

    def share_memory(self):
        for group in self.param_groups:
            for p in group['params']:
                state = self.state[p]
                state['sum'].share_memory_()

    def step(self, closure=None):
        """Performs a single optimization step.

        Arguments:
            closure (callable, optional): A closure that reevaluates the model
                and returns the loss.
        """
        loss = None
        if closure is not None:
            loss = closure()

        for group in self.param_groups:
            for p in group['params']:
                if p.grad is None:
                    continue

                grad = p.grad.data
                state = self.state[p]

                state['step'] += 1

                if group['weight_decay'] != 0:
                    if p.grad.data.is_sparse:
                        raise RuntimeError("weight_decay option is not compatible with sparse gradients ")
                    grad = grad.add(group['weight_decay'], p.data)

                clr = group['lr'] / (1 + (state['step'] - 1) * group['lr_decay'])

                if p.grad.data.is_sparse:
                    grad = grad.coalesce()  # the update is non-linear so indices must be unique
                    grad_indices = grad._indices()
                    grad_values = grad._values()
                    size = torch.Size([x for x in grad.size()])

                    def make_sparse(values):
                        constructor = type(p.grad.data)
                        if grad_indices.dim() == 0 or values.dim() == 0:
                            return constructor()
                        return constructor(grad_indices, values, size)
                    state['sum'].add_(make_sparse(grad_values.pow(2)))
                    std = state['sum']._sparse_mask(grad)
                    std_values = std._values().sqrt_().add_(1e-10)
                    p.data.add_(-clr, make_sparse(grad_values / std_values))
                else:
                    state['sum'].addcmul_(1, grad, grad)
                    std = state['sum'].sqrt().add_(1e-10)
                    p.data.addcdiv_(-clr, grad, std)

        return loss

In [26]:
def get_optimizer(name, parameters, lr, l2=0):
    if name == 'sgd':
        return torch.optim.SGD(parameters, lr=lr, weight_decay=l2)
    elif name in ['adagrad', 'myadagrad']:
        # use my own adagrad to allow for init accumulator value
        return MyAdagrad(parameters, lr=lr, init_accu_value=0.1, weight_decay=l2)
    elif name == 'adam':
        return torch.optim.Adam(parameters, weight_decay=l2, lr=lr, betas=(0.9, 0.98)) # use default lr
    elif name == 'adamax':
        return torch.optim.Adamax(parameters, weight_decay=l2) # use default lr
    elif name == 'adadelta':
        return torch.optim.Adadelta(parameters, lr=lr, weight_decay=l2)
    else:
        raise Exception("Unsupported optimizer: {}".format(name))

In [27]:
def change_lr(optimizer, new_lr):
    for param_group in optimizer.param_groups:
        param_group['lr'] = new_lr

In [28]:
def flatten_indices(seq_lens, width):
    flat = []
    for i, l in enumerate(seq_lens):
        for j in range(l):
            flat.append(i * width + j)
    return flat

In [29]:
def set_cuda(var, cuda):
    if cuda:
        return var.cuda()
    return var

In [30]:
def keep_partial_grad(grad, topk):
    """
    Keep only the topk rows of grads.
    """
    assert topk < grad.size(0)
    grad.data[topk:].zero_()
    return grad

In [31]:
def save(model, optimizer, opt, filename):
    params = {
        'model': model.state_dict(),
        'optimizer': optimizer.state_dict(),
        'config': opt
    }
    try:
        torch.save(params, filename)
    except BaseException:
        print("[ Warning: model saving failed. ]")

In [32]:
def load(model, optimizer, filename):
    try:
        dump = torch.load(filename)
    except BaseException:
        print("[ Fail: model loading failed. ]")
    if model is not None:
        model.load_state_dict(dump['model'])
    if optimizer is not None:
        optimizer.load_state_dict(dump['optimizer'])
    opt = dump['config']
    return model, optimizer, opt

In [33]:
def load_config(filename):
    try:
        dump = torch.load(filename)
    except BaseException:
        print("[ Fail: model loading failed. ]")
    return dump['config']

# Trainer

In [34]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import pdb
import numpy as np

In [35]:
class Trainer(object):
    def __init__(self, opt):
        raise NotImplementedError

    def update(self, batch):
        raise NotImplementedError

    def predict(self, batch):
        raise NotImplementedError

    def update_lr(self, new_lr):  # here should change
        change_lr(self.optimizer, new_lr)

    def load(self, filename):
        try:
            checkpoint = torch.load(filename)
        except BaseException:
            print("Cannot load model from {}".format(filename))
            exit()
        self.model.load_state_dict(checkpoint['model'])
        self.opt = checkpoint['config']

    def save(self, filename, epoch):
        params = {
            'model': self.model.state_dict(),
            'config': self.opt,
        }
        try:
            torch.save(params, filename)
            print("model saved to {}".format(filename))
        except BaseException:
            print("[Warning: Saving failed... continuing anyway.]")

In [36]:
class CDSRTrainer(Trainer):
    def __init__(self, opt, adj = None, adj_single = None):
        self.opt = opt
        if opt["model"] == "C2DSR":
            self.model = C2DSR(opt, adj, adj_single)
        else:
            print("please select a valid model")
            exit(0)

        self.mi_loss = 0
        self.BCE_criterion = nn.BCEWithLogitsLoss()
        self.CS_criterion = nn.CrossEntropyLoss(reduction='none')
        if opt['cuda']:
            self.model.cuda()
            self.BCE_criterion.cuda()
            self.CS_criterion.cuda()
        self.optimizer = get_optimizer(opt['optim'], self.model.parameters(), opt['lr'])

    def get_dot_score(self, A_embedding, B_embedding):
        output = (A_embedding * B_embedding).sum(dim=-1)
        return output

    def unpack_batch_predict(self, batch):
        if self.opt["cuda"]:
            inputs = [Variable(b.cuda()) for b in batch]
            seq = inputs[0]
            x_seq = inputs[1]
            y_seq = inputs[2]
            position = inputs[3]
            x_position = inputs[4]
            y_position = inputs[5]
            X_last = inputs[6]
            Y_last = inputs[7]
            XorY = inputs[8]
            ground_truth = inputs[9]
            neg_list = inputs[10]
        else:
            inputs = [Variable(b) for b in batch]
            seq = inputs[0]
            x_seq = inputs[1]
            y_seq = inputs[2]
            position = inputs[3]
            x_position = inputs[4]
            y_position = inputs[5]
            X_last = inputs[6]
            Y_last = inputs[7]
            XorY = inputs[8]
            ground_truth = inputs[9]
            neg_list = inputs[10]
        return seq, x_seq, y_seq, position, x_position, y_position, X_last, Y_last, XorY, ground_truth, neg_list

    def unpack_batch(self, batch):
        if self.opt["cuda"]:
            inputs = [Variable(b.cuda()) for b in batch]
            seq = inputs[0]
            x_seq = inputs[1]
            y_seq = inputs[2]
            position = inputs[3]
            x_position = inputs[4]
            y_position = inputs[5]
            ground = inputs[6]
            share_x_ground = inputs[7]
            share_y_ground = inputs[8]
            x_ground = inputs[9]
            y_ground = inputs[10]
            ground_mask = inputs[11]
            share_x_ground_mask = inputs[12]
            share_y_ground_mask = inputs[13]
            x_ground_mask = inputs[14]
            y_ground_mask = inputs[15]
            corru_x = inputs[16]
            corru_y = inputs[17]
        else:
            inputs = [Variable(b) for b in batch]
            seq = inputs[0]
            x_seq = inputs[1]
            y_seq = inputs[2]
            position = inputs[3]
            x_position = inputs[4]
            y_position = inputs[5]
            ground = inputs[6]
            share_x_ground = inputs[7]
            share_y_ground = inputs[8]
            x_ground = inputs[9]
            y_ground = inputs[10]
            ground_mask = inputs[11]
            share_x_ground_mask = inputs[12]
            share_y_ground_mask = inputs[13]
            x_ground_mask = inputs[14]
            y_ground_mask = inputs[15]
            corru_x = inputs[16]
            corru_y = inputs[17]
        return seq, x_seq, y_seq, position, x_position, y_position, ground, share_x_ground, share_y_ground, x_ground, y_ground, ground_mask, share_x_ground_mask, share_y_ground_mask, x_ground_mask, y_ground_mask, corru_x, corru_y

#     def HingeLoss(self, pos, neg):
#         gamma = torch.tensor(self.opt["margin"])
#         if self.opt["cuda"]:
#             gamma = gamma.cuda()
#         return F.relu(gamma - pos + neg).mean()
    
    def HingeLoss(self, pos, neg):
        pos_prob = torch.sigmoid(pos)
        neg_prob = 1 - torch.sigmoid(neg)

        logits = torch.cat((pos_prob, neg_prob), dim=1)
        labels = torch.ones_like(pos)  

        if self.opt["cuda"]:
            criterion = F.cross_entropy(reduction='mean').cuda()

        return criterion(logits, labels)


    def train_batch(self, batch):
        self.model.train()
        self.optimizer.zero_grad()
        self.model.graph_convolution()

        seq, x_seq, y_seq, position, x_position, y_position, ground, share_x_ground, share_y_ground, x_ground, y_ground, ground_mask, share_x_ground_mask, share_y_ground_mask, x_ground_mask, y_ground_mask, corru_x, corru_y = self.unpack_batch(batch)
        seqs_fea, x_seqs_fea, y_seqs_fea = self.model(seq, x_seq, y_seq, position, x_position, y_position)

        corru_x_fea = self.model.false_forward(corru_x, position)
        corru_y_fea = self.model.false_forward(corru_y, position)

        x_mask = x_ground_mask.float().sum(-1).unsqueeze(-1).repeat(1,x_ground_mask.size(-1))
        x_mask = 1 / x_mask
        x_mask = x_ground_mask * x_mask # for mean
        x_mask = x_mask.unsqueeze(-1).repeat(1,1,seqs_fea.size(-1))
        r_x_fea = (x_seqs_fea * x_mask).sum(1)

        y_mask = y_ground_mask.float().sum(-1).unsqueeze(-1).repeat(1, y_ground_mask.size(-1))
        y_mask = 1 / y_mask
        y_mask = y_ground_mask * y_mask # for mean
        y_mask = y_mask.unsqueeze(-1).repeat(1,1,seqs_fea.size(-1))
        r_y_fea = (y_seqs_fea * y_mask).sum(1)


        real_x_fea = (seqs_fea * x_mask).sum(1)
        real_y_fea = (seqs_fea * y_mask).sum(1)
        x_false_fea = (corru_x_fea * x_mask).sum(1)
        y_false_fea = (corru_y_fea * y_mask).sum(1)


        real_x_score = self.model.D_X(r_x_fea, real_y_fea) # the cross-domain infomax
        false_x_score = self.model.D_X(r_x_fea, y_false_fea)

        real_y_score = self.model.D_Y(r_y_fea, real_x_fea)
        false_y_score = self.model.D_Y(r_y_fea, x_false_fea)

        pos_label = torch.ones_like(real_x_score).cuda()
        neg_label = torch.zeros_like(false_x_score).cuda()
        x_mi_real = self.BCE_criterion(real_x_score, pos_label)
        x_mi_false = self.BCE_criterion(false_x_score, neg_label)
        x_mi_loss = x_mi_real + x_mi_false

        y_mi_real = self.BCE_criterion(real_y_score, pos_label)
        y_mi_false = self.BCE_criterion(false_y_score, neg_label)
        y_mi_loss = y_mi_real + y_mi_false

        used = 10
        ground = ground[:,-used:]
        ground_mask = ground_mask[:, -used:]
        share_x_ground = share_x_ground[:, -used:]
        share_x_ground_mask = share_x_ground_mask[:, -used:]
        share_y_ground = share_y_ground[:, -used:]
        share_y_ground_mask = share_y_ground_mask[:, -used:]
        x_ground = x_ground[:, -used:]
        x_ground_mask = x_ground_mask[:, -used:]
        y_ground = y_ground[:, -used:]
        y_ground_mask = y_ground_mask[:, -used:]


        share_x_result =  self.model.lin_X(seqs_fea[:,-used:]) # b * seq * X_num
        share_y_result = self.model.lin_Y(seqs_fea[:, -used:])  # b * seq * Y_num
        share_pad_result = self.model.lin_PAD(seqs_fea[:, -used:])  # b * seq * 1
        share_trans_x_result = torch.cat((share_x_result, share_pad_result), dim=-1)
        share_trans_y_result = torch.cat((share_y_result, share_pad_result), dim=-1)


        specific_x_result = self.model.lin_X(seqs_fea[:,-used:] + x_seqs_fea[:, -used:])  # b * seq * X_num
        specific_x_pad_result = self.model.lin_PAD(x_seqs_fea[:, -used:])  # b * seq * 1
        specific_x_result = torch.cat((specific_x_result, specific_x_pad_result), dim=-1)

        specific_y_result = self.model.lin_Y(seqs_fea[:,-used:] + y_seqs_fea[:, -used:])  # b * seq * Y_num
        specific_y_pad_result = self.model.lin_PAD(y_seqs_fea[:, -used:])  # b * seq * 1
        specific_y_result = torch.cat((specific_y_result, specific_y_pad_result), dim=-1)

        x_share_loss = self.CS_criterion(
            share_trans_x_result.reshape(-1, self.opt["source_item_num"] + 1),
            share_x_ground.reshape(-1))  # b * seq
        y_share_loss = self.CS_criterion(
            share_trans_y_result.reshape(-1, self.opt["target_item_num"] + 1),
            share_y_ground.reshape(-1))  # b * seq
        x_loss = self.CS_criterion(
            specific_x_result.reshape(-1, self.opt["source_item_num"] + 1),
            x_ground.reshape(-1))  # b * seq
        y_loss = self.CS_criterion(
            specific_y_result.reshape(-1, self.opt["target_item_num"] + 1),
            y_ground.reshape(-1))  # b * seq

        x_share_loss = (x_share_loss * (share_x_ground_mask.reshape(-1))).mean()
        y_share_loss = (y_share_loss * (share_y_ground_mask.reshape(-1))).mean()
        x_loss = (x_loss * (x_ground_mask.reshape(-1))).mean()
        y_loss = (y_loss * (y_ground_mask.reshape(-1))).mean()

        loss = self.opt["lambda"]*(x_share_loss + y_share_loss + x_loss + y_loss) + (1 - self.opt["lambda"]) * (x_mi_loss + y_mi_loss)

        self.mi_loss += (1 - self.opt["lambda"]) * (x_mi_loss.item() + y_mi_loss.item())
        loss.backward()
        self.optimizer.step()

        return loss.item()

    def test_batch(self, batch):
        seq, x_seq, y_seq, position, x_position, y_position, X_last, Y_last, XorY, ground_truth, neg_list = self.unpack_batch_predict(batch)
        seqs_fea, x_seqs_fea, y_seqs_fea = self.model(seq, x_seq, y_seq, position, x_position, y_position)

        X_pred = []
        Y_pred = []
        for id, fea in enumerate(seqs_fea): # b * s * f
            if XorY[id] == 0:
                share_fea = seqs_fea[id, -1]
                specific_fea = x_seqs_fea[id, X_last[id]]
                X_score = self.model.lin_X(share_fea + specific_fea).squeeze(0)
                cur = X_score[ground_truth[id]]
                score_larger = (X_score[neg_list[id]] > (cur + 0.00001)).data.cpu().numpy()
                true_item_rank = np.sum(score_larger) + 1
                X_pred.append(true_item_rank)

            else :
                share_fea = seqs_fea[id, -1]
                specific_fea = y_seqs_fea[id, Y_last[id]]
                Y_score = self.model.lin_Y(share_fea + specific_fea).squeeze(0)
                cur = Y_score[ground_truth[id]]
                score_larger = (Y_score[neg_list[id]] > (cur + 0.00001)).data.cpu().numpy()
                true_item_rank = np.sum(score_larger) + 1
                Y_pred.append(true_item_rank)

        return X_pred, Y_pred

# Train_Rec

In [37]:
import os
import sys
from datetime import datetime
import time
import numpy as np
import random
import argparse
from shutil import copyfile
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import json
import codecs
import tqdm
import pdb

In [38]:
parser = argparse.ArgumentParser()

parser.add_argument('-f')
# dataset part
parser.add_argument('--data_dir', type=str, default='Entertainment-Education', help='Food-Kitchen, Movie-Book')

# model part
parser.add_argument('--model', type=str, default="C2DSR", help='model name')
parser.add_argument('--hidden_units', type=int, default=256, help='lantent dim.')
parser.add_argument('--num_blocks', type=int, default=2, help='lantent dim.')
parser.add_argument('--num_heads', type=int, default=1, help='lantent dim.')
parser.add_argument('--GNN', type=int, default=1, help='GNN depth.')
parser.add_argument('--dropout', type=float, default=0.2, help='dropout rate.')
parser.add_argument('--optim', choices=['sgd', 'adagrad', 'adam', 'adamax'], default='adagrad',
                    help='Optimizer: sgd, adagrad, adam or adamax.')
parser.add_argument('--lr', type=float, default=0.001, help='Applies to sgd and adagrad.')
parser.add_argument('--lr_decay', type=float, default=1, help='Learning rate decay rate.')
parser.add_argument('--weight_decay', type=float, default=5e-4, help='Weight decay (L2 loss on parameters).')
parser.add_argument('--decay_epoch', type=int, default=5, help='Decay learning rate after this epoch.')
parser.add_argument('--max_grad_norm', type=float, default=5.0, help='Gradient clipping.')
parser.add_argument('--leakey', type=float, default=0.1)
parser.add_argument('--maxlen', type=int, default=15)
parser.add_argument('--cpu', action='store_true', help='Ignore CUDA.')
parser.add_argument('--cuda', type=bool, default=torch.cuda.is_available())
parser.add_argument('--lambda', type=float, default=0.7)

# train part
parser.add_argument('--num_epoch', type=int, default=100, help='Number of total training epochs.')
parser.add_argument('--batch_size', type=int, default=256, help='Training batch size.')
parser.add_argument('--log_step', type=int, default=200, help='Print log every k steps.')
parser.add_argument('--log', type=str, default='logs.txt', help='Write training log to file.')
parser.add_argument('--save_epoch', type=int, default=100, help='Save model checkpoints every k epochs.')
parser.add_argument('--save_dir', type=str, default='./saved_models', help='Root dir for saving models.')
parser.add_argument('--id', type=str, default='00', help='Model ID under which to save models.')
parser.add_argument('--seed', type=int, default=2040)
parser.add_argument('--load', dest='load', action='store_true', default=False,  help='Load pretrained model.')
parser.add_argument('--model_file', type=str, help='Filename of the pretrained model.')
parser.add_argument('--info', type=str, default='', help='Optional info for the experiment.')
parser.add_argument('--undebug', action='store_false', default=True)

_StoreFalseAction(option_strings=['--undebug'], dest='undebug', nargs=0, const=False, default=True, type=None, choices=None, required=False, help=None, metavar=None)

In [39]:
def seed_everything(seed=1111):
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [40]:
args = parser.parse_args()
if args.cpu:
    args.cuda = False
elif args.cuda:
    torch.cuda.manual_seed(args.seed)
init_time = time.time()
# make opt
opt = vars(args)

seed_everything(opt["seed"])

model_id = opt['id'] if len(opt['id']) > 1 else '0' + opt['id']
model_save_dir = opt['save_dir'] + '/' + model_id
opt['model_save_dir'] = model_save_dir
ensure_dir(model_save_dir, verbose=True)
# save config
save_config(opt, model_save_dir + '/config.json', verbose=True)
file_logger = FileLogger(model_save_dir + '/' + opt['log'],
                                header="# epoch\ttrain_loss\tdev_loss\tdev_score\tbest_dev_score")

Directory ./saved_models/00 do not exist; creating...
Config saved to file ./saved_models/00/config.json


In [41]:
# print model info
print_config(opt)

if opt["undebug"]:
    pass
    # opt["cuda"] = False
    # opt["cpu"] = True

print("Loading data from {} with batch size {}...".format(opt['data_dir'], opt['batch_size']))
train_batch = DataLoader(opt['data_dir'], opt['batch_size'], opt, evaluation = -1)
valid_batch = DataLoader(opt['data_dir'], opt["batch_size"], opt, evaluation = 2)
test_batch = DataLoader(opt['data_dir'], opt["batch_size"], opt, evaluation = 1)
print("Data loading done!")

opt["itemnum"] = opt["source_item_num"] + opt["target_item_num"] + 1


Running with the following configs:
	f : /root/.local/share/jupyter/runtime/kernel-e9726718-a74a-4680-b407-7667c7c071ff.json
	data_dir : Entertainment-Education
	model : C2DSR
	hidden_units : 256
	num_blocks : 2
	num_heads : 1
	GNN : 1
	dropout : 0.2
	optim : adagrad
	lr : 0.001
	lr_decay : 1
	weight_decay : 0.0005
	decay_epoch : 5
	max_grad_norm : 5.0
	leakey : 0.1
	maxlen : 15
	cpu : False
	cuda : True
	lambda : 0.7
	num_epoch : 100
	batch_size : 256
	log_step : 200
	log : logs.txt
	save_epoch : 100
	save_dir : ./saved_models
	id : 00
	seed : 2040
	load : False
	model_file : None
	info : 
	undebug : True
	model_save_dir : ./saved_models/00


Loading data from Entertainment-Education with batch size 256...
pass sequence x
Data loading done!


In [42]:
filename = opt["data_dir"]
train_data = "/kaggle/input/cross-domain-recommendation/dataset/" + filename + "/traindata_new.txt"
G = GraphMaker(opt, train_data)
adj, adj_single = G.adj, G.adj_single
print("graph loaded!")

if opt["cuda"]:
    adj = adj.cuda()
    adj_single = adj_single.cuda()

/tmp/ipykernel_35/2768315390.py:4: RuntimeWarning: divide by zero encountered in power
  r_inv = np.power(rowsum, -1).flatten()
/tmp/ipykernel_35/3650411315.py:8: UserWarning: torch.sparse.SparseTensor(indices, values, shape, *, device=) is deprecated.  Please use torch.sparse_coo_tensor(indices, values, shape, dtype=, device=). (Triggered internally at /usr/local/src/pytorch/torch/csrc/utils/tensor_new.cpp:605.)
  return torch.sparse.FloatTensor(indices, values, shape)


real graph loaded!
graph loaded!


In [43]:
# model
if not opt['load']:
    trainer = CDSRTrainer(opt, adj, adj_single)
else:
    exit(0)

global_step = 0
current_lr = opt["lr"]
format_str = '{}: step {}/{} (epoch {}/{}), loss = {:.6f} ({:.3f} sec/epoch), lr: {:.6f}'
num_batch = len(train_batch)
max_steps = opt['num_epoch'] * num_batch

print("Start training:")

begin_time = time.time()
X_dev_score_history=[0]
Y_dev_score_history=[0]

Start training:


In [44]:
# start training
for epoch in range(1, opt['num_epoch'] + 1):
    train_loss = 0
    epoch_start_time = time.time()
    trainer.mi_loss = 0
    for batch in train_batch:
        global_step += 1
        loss = trainer.train_batch(batch)
        train_loss += loss

    duration = time.time() - epoch_start_time
    print(format_str.format(datetime.now(), global_step, max_steps, epoch, \
                                    opt['num_epoch'], train_loss/num_batch, duration, current_lr))
    print("mi:", trainer.mi_loss/num_batch)

    if epoch % 5:
        continue

    # eval model
    print("Evaluating on dev set...")

    trainer.model.eval()
    trainer.model.graph_convolution()


    def cal_test_score(predictions):
        MRR=0.0
        HR_1 = 0.0
        HR_5 = 0.0
        HR_10 = 0.0
        NDCG_5 = 0.0
        NDCG_10 = 0.0
        valid_entity = 0.0
        # pdb.set_trace()
        for pred in predictions:
            valid_entity += 1
            MRR += 1 / pred
            if pred <= 1:
                HR_1 += 1
            if pred <= 5:
                NDCG_5 += 1 / np.log2(pred + 1)
                HR_5 += 1
            if pred <= 10:
                NDCG_10 += 1 / np.log2(pred + 1)
                HR_10 += 1
            if valid_entity % 100 == 0:
                print('.', end='')
        return MRR/valid_entity, NDCG_5 / valid_entity, NDCG_10 / valid_entity, HR_1 / valid_entity, HR_5 / valid_entity, HR_10 / valid_entity

    def get_evaluation_result(evaluation_batch):
        X_pred = []
        Y_pred = []
        for i, batch in enumerate(evaluation_batch):
            X_predictions, Y_predictions = trainer.test_batch(batch)
            X_pred = X_pred + X_predictions
            Y_pred = Y_pred + Y_predictions

        return X_pred, Y_pred


    val_X_pred, val_Y_pred = get_evaluation_result(valid_batch)
    val_X_MRR, val_X_NDCG_5, val_X_NDCG_10, val_X_HR_1, val_X_HR_5, val_X_HR_10 = cal_test_score(val_X_pred)
    val_Y_MRR, val_Y_NDCG_5, val_Y_NDCG_10, val_Y_HR_1, val_Y_HR_5, val_Y_HR_10 = cal_test_score(val_Y_pred)

    print("")
    print('val epoch:%d, time: %f(s), X (MRR: %.4f, NDCG@10: %.4f, HR@10: %.4f), Y (MRR: %.4f, NDCG@10: %.4f, HR@10: %.4f)'
          % (epoch, time.time() - begin_time, val_X_MRR, val_X_NDCG_10, val_X_HR_10, val_Y_MRR, val_Y_NDCG_10, val_Y_HR_10))

    if val_X_MRR > max(X_dev_score_history) or val_Y_MRR > max(Y_dev_score_history):
        test_X_pred, test_Y_pred = get_evaluation_result(test_batch)
        test_X_MRR, test_X_NDCG_5, test_X_NDCG_10, test_X_HR_1, test_X_HR_5, test_X_HR_10 = cal_test_score(test_X_pred)
        test_Y_MRR, test_Y_NDCG_5, test_Y_NDCG_10, test_Y_HR_1, test_Y_HR_5, test_Y_HR_10 = cal_test_score(test_Y_pred)

        print("")
        if val_X_MRR > max(X_dev_score_history):
            print("X best!")
            print([test_X_MRR, test_X_NDCG_5, test_X_NDCG_10, test_X_HR_1, test_X_HR_5, test_X_HR_10])

        if val_Y_MRR > max(Y_dev_score_history):
            print("Y best!")
            print([test_Y_MRR, test_Y_NDCG_5, test_Y_NDCG_10, test_Y_HR_1, test_Y_HR_5, test_Y_HR_10])

    X_dev_score_history.append(val_X_MRR)
    Y_dev_score_history.append(val_Y_MRR)

/tmp/ipykernel_35/2765941852.py:77: UserWarning: This overload of addcmul_ is deprecated:
	addcmul_(Number value, Tensor tensor1, Tensor tensor2)
Consider using one of the following signatures instead:
	addcmul_(Tensor tensor1, Tensor tensor2, *, Number value) (Triggered internally at /usr/local/src/pytorch/torch/csrc/utils/python_arg_parser.cpp:1519.)
  state['sum'].addcmul_(1, grad, grad)


2024-03-19 09:03:45.003875: step 472/47200 (epoch 1/100), loss = 13.263966 (172.154 sec/epoch), lr: 0.001000
mi: 0.7943236586148454
2024-03-19 09:06:40.057514: step 944/47200 (epoch 2/100), loss = 12.950979 (175.053 sec/epoch), lr: 0.001000
mi: 0.7259552791714668
2024-03-19 09:09:34.918771: step 1416/47200 (epoch 3/100), loss = 12.337447 (174.861 sec/epoch), lr: 0.001000
mi: 0.6789788725770127
2024-03-19 09:12:29.949845: step 1888/47200 (epoch 4/100), loss = 11.514235 (175.031 sec/epoch), lr: 0.001000
mi: 0.6589505904054235
2024-03-19 09:15:24.885353: step 2360/47200 (epoch 5/100), loss = 11.028319 (174.935 sec/epoch), lr: 0.001000
mi: 0.6380613603202969
Evaluating on dev set...
.....................................................................
val epoch:5, time: 876.772353(s), X (MRR: 0.2182, NDCG@10: 0.2686, HR@10: 0.4597), Y (MRR: 0.2874, NDCG@10: 0.3236, HR@10: 0.4559)
...................................................................
X best!
[0.22963391819446038, 0.23961635964